In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1 = pd.read_csv('data/articles1.csv')
df_1

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...,...,...,...,...,...,...
49995,53287,73465,"Rex Tillerson Says Climate Change Is Real, but …",Atlantic,Robinson Meyer,2017-01-11,2017.0,1.0,NaN,"As chairman and CEO of ExxonMobil, Rex Tillers..."
49996,53288,73466,The Biggest Intelligence Questions Raised by t...,Atlantic,Amy Zegart,2017-01-11,2017.0,1.0,NaN,I’ve spent nearly 20 years looking at intellig...
49997,53289,73467,Trump Announces Plan That Does Little to Resol...,Atlantic,Jeremy Venook,2017-01-11,2017.0,1.0,NaN,Donald Trump will not be taking necessary st...
49998,53290,73468,Dozens of For-Profit Colleges Could Soon Close,Atlantic,Emily DeRuy,2017-01-11,2017.0,1.0,NaN,Dozens of colleges could be forced to close ...


In [3]:
titles = df_1['title'].array
papers = df_1['content'].array

In [4]:
titles[1]

'Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times'

In [11]:
papers[1]

'After the bullet shells get counted, the blood dries and the votive candles burn out, people peer down from   windows and see crime scenes gone cold: a band of yellow police tape blowing in the breeze. The South Bronx, just across the Harlem River from Manhattan and once shorthand for urban dysfunction, still suffers violence at levels long ago slashed in many other parts of New York City. And yet the city’s efforts to fight it remain splintered, underfunded and burdened by scandal. In the 40th Precinct, at the southern tip of the Bronx, as in other poor, minority neighborhoods across the country, people long hounded for   infractions are crying out for more protection against grievous injury or death. By September, four of every five shootings in the precinct this year were unsolved. Out of the city’s 77 precincts, the 40th has the highest murder rate but the fewest detectives per violent crime, reflecting disparities in staffing that hit hardest in some neighborhoods outside Manhatt

We perform some basic text wrangling or preprocessing before diving into topic modeling. We keep things simple here and perform tokenization, lemmatizing nouns, and removing stopwords and any terms having a single character.


In [12]:
%%time
import nltk

stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalise_corpus(papers, titles):
    norm_papers = []
    pre_papers = []
    pre_titles = []
    for i in range(len(papers)):
        paper = papers[i]
        title = titles[i]

        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))

        if paper_tokens:
            norm_papers.append(paper_tokens)
            pre_papers.append(paper)
            pre_titles.append(title)

    return norm_papers, pre_papers, pre_titles


def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
    return norm_papers


# norm_papers = normalize_corpus(papers)

norm_papers, pre_papers, pre_titles = normalise_corpus(papers, titles)

print(len(norm_papers))

49933
CPU times: user 2min 26s, sys: 689 ms, total: 2min 26s
Wall time: 2min 26s


In [13]:
len(pre_papers), len(pre_titles)

(49933, 49933)

### topic moedlling with gensim

In [14]:
import gensim

# higher threshold fewer phrases
bigram = gensim.models.Phrases(norm_papers, min_count=20, threshold=20, delimiter=b'_')

bigram_model = gensim.models.phrases.Phraser(bigram)

# sample demonstration
print(bigram_model[norm_papers[0]][:50])

['washington', 'congressional', 'republican', 'new', 'fear', 'come', 'health_care', 'lawsuit', 'obama_administration', 'might', 'win', 'incoming', 'trump', 'administration', 'could', 'choose', 'longer', 'defend', 'executive_branch', 'suit', 'challenge', 'administration', 'authority', 'spend', 'billion_dollar', 'health_insurance', 'subsidy', 'american', 'handing', 'house', 'republican', 'big', 'victory', 'issue', 'sudden', 'loss', 'disputed', 'subsidy', 'could_conceivably', 'cause', 'health_care', 'program', 'implode', 'leaving', 'million', 'people', 'without', 'access', 'health_insurance', 'republican']


In [15]:
norm_corpus_bigrams = [bigram_model[doc] for doc in norm_papers]

# Create a dictionary representation of the documents.
dictionary = gensim.corpora.Dictionary(norm_corpus_bigrams)

print('Sample word to number mappings:', list(dictionary.items())[:15])
print('\n Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, 'access'), (1, 'acknowledge'), (2, 'act'), (3, 'administration'), (4, 'advocate'), (5, 'affordable_care'), (6, 'ally'), (7, 'american'), (8, 'angering'), (9, 'annual'), (10, 'another'), (11, 'anticipated'), (12, 'anticipating'), (13, 'appeal'), (14, 'appealed')]

 Total Vocabulary Size: 170226


In [16]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.6)
print('Total Vocabulary Size:', len(dictionary))

Total Vocabulary Size: 33588


In [17]:
# Transforming corpus into bag of words vectors
bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigrams]
print(bow_corpus[1][:50])

[(0, 1), (4, 1), (6, 1), (10, 3), (21, 2), (31, 18), (32, 1), (34, 1), (39, 4), (40, 1), (45, 1), (57, 5), (59, 2), (64, 1), (66, 1), (68, 1), (81, 1), (84, 1), (85, 3), (88, 2), (90, 2), (93, 3), (97, 2), (99, 3), (100, 3), (102, 1), (109, 1), (110, 1), (112, 3), (116, 9), (124, 1), (136, 1), (138, 1), (140, 2), (141, 6), (142, 2), (143, 1), (148, 1), (157, 4), (159, 2), (160, 2), (162, 2), (163, 4), (165, 22), (167, 3), (168, 4), (171, 4), (177, 4), (180, 1), (181, 1)]


In [19]:
# viewing actual terms and their counts
print([(dictionary[idx] , freq) for idx, freq in bow_corpus[1][:50]])

[('access', 1), ('advocate', 1), ('ally', 1), ('another', 3), ('authority', 2), ('case', 18), ('cause', 1), ('challenge', 1), ('come', 4), ('comment', 1), ('confident', 1), ('could', 5), ('court', 2), ('decision', 1), ('defend', 1), ('desperate', 1), ('drop', 1), ('effect', 1), ('effort', 3), ('end', 2), ('era', 2), ('even_though', 3), ('fear', 2), ('fight', 3), ('find', 3), ('found', 1), ('given', 1), ('government', 1), ('handle', 3), ('house', 9), ('inclined', 1), ('judge', 1), ('lack', 1), ('late', 2), ('later', 6), ('law', 2), ('lawsuit', 1), ('leaving', 1), ('many', 4), ('may', 2), ('might', 2), ('money', 2), ('month', 4), ('mr', 22), ('never', 3), ('new', 4), ('office', 4), ('part', 4), ('paying', 1), ('payment', 1)]


### LDA

In [20]:
%%time

TOTAL_TOPICS = 10
lda_model = gensim.models.LdaModel(corpus=bow_corpus, id2word=dictionary,
                                   chunksize=1740, alpha="auto", eta="auto",
                                   random_state=42, iterations=500, num_topics=TOTAL_TOPICS,
                                   passes=20, eval_every=None)

CPU times: user 1h 3min 17s, sys: 20.8 s, total: 1h 3min 38s
Wall time: 12min 57s


In [21]:
for topic_id, topic in lda_model.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

Topic #1:
0.036*"trump" + 0.020*"president" + 0.012*"obama" + 0.009*"would" + 0.009*"white_house" + 0.009*"russia" + 0.007*"country" + 0.006*"administration" + 0.006*"american" + 0.006*"mr" + 0.006*"russian" + 0.005*"leader" + 0.005*"also" + 0.005*"united_state" + 0.005*"donald_trump" + 0.004*"official" + 0.004*"former" + 0.004*"meeting" + 0.004*"government" + 0.004*"deal"

Topic #2:
0.014*"police" + 0.009*"city" + 0.008*"family" + 0.007*"two" + 0.007*"officer" + 0.006*"people" + 0.006*"home" + 0.006*"day" + 0.005*"man" + 0.005*"child" + 0.005*"according" + 0.004*"told" + 0.004*"three" + 0.004*"shooting" + 0.004*"cnn" + 0.004*"time" + 0.004*"car" + 0.004*"state" + 0.003*"student" + 0.003*"death"

Topic #3:
0.047*"trump" + 0.021*"clinton" + 0.014*"campaign" + 0.013*"republican" + 0.012*"donald_trump" + 0.009*"say" + 0.009*"election" + 0.009*"candidate" + 0.008*"hillary_clinton" + 0.008*"voter" + 0.007*"party" + 0.007*"people" + 0.007*"vote" + 0.006*"state" + 0.006*"president" + 0.005*"c

In [22]:
topics_coherences = lda_model.top_topics(bow_corpus, topn=20)
avg_coherence_score = np.mean([item[1] for item in topics_coherences])
print('Avg. Coherence Score:', avg_coherence_score)

Avg. Coherence Score: -1.3628021129807837


In [23]:
topics_with_wts = [item[0] for item in topics_coherences]
print('LDA Topics with Weights')
print('='*50)
for idx, topic in enumerate(topics_with_wts):
    print('Topic #'+str(idx+1)+':')
    print([(term, round(wt, 3)) for wt, term in topic])
    print()

LDA Topics with Weights
Topic #1:
[('people', 0.012), ('like', 0.012), ('get', 0.01), ('say', 0.01), ('would', 0.008), ('think', 0.008), ('thing', 0.008), ('time', 0.008), ('know', 0.008), ('going', 0.007), ('way', 0.007), ('want', 0.006), ('go', 0.006), ('could', 0.005), ('make', 0.005), ('even', 0.005), ('really', 0.005), ('see', 0.005), ('back', 0.005), ('much', 0.005)]

Topic #2:
[('trump', 0.047), ('clinton', 0.021), ('campaign', 0.014), ('republican', 0.013), ('donald_trump', 0.012), ('say', 0.009), ('election', 0.009), ('candidate', 0.009), ('hillary_clinton', 0.008), ('voter', 0.008), ('party', 0.007), ('people', 0.007), ('vote', 0.007), ('state', 0.006), ('president', 0.006), ('cruz', 0.005), ('democrat', 0.005), ('would', 0.005), ('poll', 0.005), ('think', 0.005)]

Topic #3:
[('trump', 0.036), ('president', 0.02), ('obama', 0.012), ('would', 0.009), ('white_house', 0.009), ('russia', 0.009), ('country', 0.007), ('administration', 0.006), ('american', 0.006), ('mr', 0.006), ('

In [24]:
print('LDA Topics without Weights')
print('='*50)
for idx, topic in enumerate(topics_with_wts):
    print('Topic #'+str(idx+1)+':')
    print([term for wt, term in topic])
    print()

LDA Topics without Weights
Topic #1:
['people', 'like', 'get', 'say', 'would', 'think', 'thing', 'time', 'know', 'going', 'way', 'want', 'go', 'could', 'make', 'even', 'really', 'see', 'back', 'much']

Topic #2:
['trump', 'clinton', 'campaign', 'republican', 'donald_trump', 'say', 'election', 'candidate', 'hillary_clinton', 'voter', 'party', 'people', 'vote', 'state', 'president', 'cruz', 'democrat', 'would', 'poll', 'think']

Topic #3:
['trump', 'president', 'obama', 'would', 'white_house', 'russia', 'country', 'administration', 'american', 'mr', 'russian', 'leader', 'also', 'united_state', 'donald_trump', 'official', 'former', 'meeting', 'government', 'deal']

Topic #4:
['attack', 'people', 'isi', 'country', 'government', 'group', 'muslim', 'syria', 'also', 'force', 'killed', 'military', 'city', 'war', 'many', 'official', 'year', 'security', 'turkey', 'two']

Topic #5:
['police', 'city', 'family', 'two', 'officer', 'people', 'home', 'day', 'man', 'child', 'according', 'told', 'three'

### evaluating model

In [25]:
cv_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus,
                                                 texts=norm_corpus_bigrams,
                                                     dictionary=dictionary,
                                                      coherence='c_v')
avg_coherence_cv = cv_coherence_model_lda.get_coherence()
umass_coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus,
                                               texts=norm_corpus_bigrams,
                                                     dictionary=dictionary,
                                                        coherence='u_mass')
avg_coherence_umass = umass_coherence_model_lda.get_coherence()
perplexity = lda_model.log_perplexity(bow_corpus)
print('Avg. Coherence Score (Cv):', avg_coherence_cv)
print('Avg. Coherence Score (UMass):', avg_coherence_umass)
print('Model Perplexity:', perplexity)

Avg. Coherence Score (Cv): 0.4611817097708893
Avg. Coherence Score (UMass): -1.3628021129807835
Model Perplexity: -8.525454085960877


### checking topics

In [26]:
topics_df = pd.DataFrame([[term for wt, term in topic]
                              for topic in topics_with_wts],
                         columns = ['Term'+str(i) for i in range(1, 21)],
                         index=['Topic '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
Term1,people,trump,trump,attack,police,would,investigation,woman,company,company
Term2,like,clinton,president,people,city,state,report,show,year,new
Term3,get,campaign,obama,isi,family,law,official,year,million,china
Term4,say,republican,would,country,two,republican,case,also,business,apple
Term5,would,donald_trump,white_house,government,officer,american,information,time,market,also
Term6,think,say,russia,group,people,bill,time,black,percent,car
Term7,thing,election,country,muslim,home,people,email,life,according,tesla
Term8,time,candidate,administration,syria,day,policy,according,new,money,could
Term9,know,hillary_clinton,american,also,man,president,fbi,first,price,google
Term10,going,voter,mr,force,child,house,also,like,number,would


In [27]:
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for wt, term in topic])
                              for topic in topics_with_wts],
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, lda_model.num_topics+1)]
                         )
topics_df

,Terms per Topic
Topic1,"people, like, get, say, would, think, thing, time, know, going, way, want, go, could, make, even, really, see, back, much"
Topic2,"trump, clinton, campaign, republican, donald_trump, say, election, candidate, hillary_clinton, voter, party, people, vote, state, president, cruz, democrat, would, poll, think"
Topic3,"trump, president, obama, would, white_house, russia, country, administration, american, mr, russian, leader, also, united_state, donald_trump, official, former, meeting, government, deal"
Topic4,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two"
Topic5,"police, city, family, two, officer, people, home, day, man, child, according, told, three, shooting, cnn, time, car, state, student, death"
Topic6,"would, state, law, republican, american, bill, people, policy, president, house, right, issue, also, plan, could, senate, government, federal, gop, court"
Topic7,"investigation, report, official, case, information, time, email, according, fbi, also, statement, told, former, comey, evidence, reported, cnn, intelligence, would, charge"
Topic8,"woman, show, year, also, time, black, life, new, first, like, story, white, film, wrote, star, book, people, movie, made, family"
Topic9,"company, year, million, business, market, percent, according, money, price, number, billion, job, also, new, investor, report, cost, study, economy, since"
Topic10,"company, new, china, apple, also, car, tesla, could, google, would, technology, system, facebook, north_korea, ceo, product, use, time, chinese, service"


### interpreting results

In [28]:
tm_results = lda_model[bow_corpus]

In [29]:
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0]
                     for topics in tm_results]
corpus_topics[:5]

[(3, 0.7365905),
 (1, 0.54163784),
 (6, 0.58427244),
 (6, 0.56559813),
 (5, 0.65246284)]

In [30]:
corpus_topics[1]

(1, 0.54163784)

In [31]:
len(papers) - len(corpus_topics)

67

In [32]:
corpus_topic_df = pd.DataFrame()
corpus_topic_df['Document'] = range(0, len(papers) - 67)
corpus_topic_df['Dominant Topic'] = [item[0]+1 for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
corpus_topic_df['Topic Desc'] = [topics_df.iloc[t[0]]['Terms per Topic'] for t in corpus_topics]
corpus_topic_df['Title'] = pre_titles
corpus_topic_df['Paper'] = pre_papers
#corpus_topic_df['Title'] = titles[:-67]

In [33]:
pd.set_option('display.max_colwidth', 200)

topic_stats_df = corpus_topic_df.groupby('Dominant Topic').agg({
                                                'Dominant Topic': {
                                                    'Doc Count': np.size,
                                                    '% Total Docs': np.size }
                                              })
topic_stats_df = topic_stats_df['Dominant Topic'].reset_index()
topic_stats_df['% Total Docs'] = topic_stats_df['% Total Docs'].apply(lambda row: round((row*100) / len(papers), 2))
topic_stats_df['Topic Desc'] = [topics_df.iloc[t]['Terms per Topic'] for t in range(len(topic_stats_df))]
topic_stats_df

/Users/alex.nim@ibm.com/opt/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


,Dominant Topic,Doc Count,% Total Docs,Topic Desc
0,1,4935,9.87,"people, like, get, say, would, think, thing, time, know, going, way, want, go, could, make, even, really, see, back, much"
1,2,4965,9.93,"trump, clinton, campaign, republican, donald_trump, say, election, candidate, hillary_clinton, voter, party, people, vote, state, president, cruz, democrat, would, poll, think"
2,3,7997,15.99,"trump, president, obama, would, white_house, russia, country, administration, american, mr, russian, leader, also, united_state, donald_trump, official, former, meeting, government, deal"
3,4,5171,10.34,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two"
4,5,6176,12.35,"police, city, family, two, officer, people, home, day, man, child, according, told, three, shooting, cnn, time, car, state, student, death"
5,6,3149,6.30,"would, state, law, republican, american, bill, people, policy, president, house, right, issue, also, plan, could, senate, government, federal, gop, court"
6,7,7282,14.56,"investigation, report, official, case, information, time, email, according, fbi, also, statement, told, former, comey, evidence, reported, cnn, intelligence, would, charge"
7,8,4113,8.23,"woman, show, year, also, time, black, life, new, first, like, story, white, film, wrote, star, book, people, movie, made, family"
8,9,1582,3.16,"company, year, million, business, market, percent, according, money, price, number, billion, job, also, new, investor, report, cost, study, economy, since"
9,10,4563,9.13,"company, new, china, apple, also, car, tesla, could, google, would, technology, system, facebook, north_korea, ceo, product, use, time, chinese, service"


### document most dominant topic with highest contribution %

In [34]:
corpus_topic_df.sort_values(by='Contribution %', ascending=False).head(10)

,Document,Dominant Topic,Contribution %,Topic Desc,Title,Paper
167,167,7,98.46,"investigation, report, official, case, information, time, email, according, fbi, also, statement, told, former, comey, evidence, reported, cnn, intelligence, would, charge",Golden Globe Award Winners 2017: The Complete List - The New York Times,"here are the movies and tv shows that received the top awards. best picture, drama: “moonlight” best picture, comedy or musical: “la la land” actress, drama: isabelle huppert, “elle” actor, drama:..."
44533,44533,4,98.42,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",Trump just scored a big win with the conservatives who killed his healthcare bill,’ ’ ’ the conservative house freedom caucus on wednesday reversed course and said it was on board with the latest iteration of a republican plan to repeal and replace obamacare. ’ ” to the ...
6474,6474,7,97.54,"investigation, report, official, case, information, time, email, according, fbi, also, statement, told, former, comey, evidence, reported, cnn, intelligence, would, charge",George Michael’s Death Is Met With Disbelief by Celebrities on Twitter - The New York Times,"george michael, the pop superstar and icon of the 1980s and ’90s, died on sunday, provoking an outpouring of fond and reaction from fans around the world, many of them celebrities in their own ..."
10057,10057,4,96.87,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",Utah Lawmakers Pass Measure Calling on Congress to Abolish U.S. Education Department,"the utah state legislature has passed a joint resolution calling for a return to federalism in general, and, in particular, the end of the federal education department. [the measure passed the uta..."
24008,24008,4,96.78,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",Supreme Court Justices Meet to Consider Taking Up Executive Amnesty Appeal - Breitbart,"the u. s. supreme court could rule as early as this week on whether or not they will hear the government’s appeal to implement president obama’s executive amnesty. [according to reuters, the supre..."
45729,45729,3,96.55,"trump, president, obama, would, white_house, russia, country, administration, american, mr, russian, leader, also, united_state, donald_trump, official, former, meeting, government, deal",Here’s the Hillary Clinton campaign’s memo spinning her New Hampshire loss,’the campaign of democratic presidential candidate hillary clinton released a memo downplaying the importance of the four early voting states after she to us sen. bernie sanders ( ).’ ’sanders wa...
22801,22801,3,96.35,"trump, president, obama, would, white_house, russia, country, administration, american, mr, russian, leader, also, united_state, donald_trump, official, former, meeting, government, deal",Watch: Sanders Claims He’s the Best Dem Candidate to Take on Donald Trump - Breitbart,"during sunday’s democratic presidential debate on nbc, sen. bernie sanders ( ) argued he would be the best candidate of he and his opponent hillary clinton to take on republican presidential don..."
9997,9997,1,96.16,"people, like, get, say, would, think, thing, time, know, going, way, want, go, could, make, even, really, see, back, much",Netanyahu: U.S.-Israel Alliance is ’About to Get Stronger’,ynetnews reports: prime minister benjamin netanyahu promised to strengthen the alliance between the us and israel on monday afternoon before taking off to washington for his first meeting with pre...
31053,31053,4,96.12,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",Supreme Court Will Hear Executive Amnesty Appeal - Breitbart,the supreme

In [35]:
pd.set_option('display.max_colwidth', 200)
(corpus_topic_df[corpus_topic_df['Document']
                 .isin([681, 9, 392, 1622, 17,
                        906, 996, 503, 13, 733])])

,Document,Dominant Topic,Contribution %,Topic Desc,Title,Paper
9,9,2,45.04,"trump, clinton, campaign, republican, donald_trump, say, election, candidate, hillary_clinton, voter, party, people, vote, state, president, cruz, democrat, would, poll, think",Calling on Angels While Enduring the Trials of Job - The New York Times,"angels are everywhere in the muñiz family’s apartment in the bronx: paintings of angels on the wall, ceramic angels flanking the ancient vcr, angels strumming lyres or blowing little golden trumpe..."
13,13,5,83.59,"police, city, family, two, officer, people, home, day, man, child, according, told, three, shooting, cnn, time, car, state, student, death",How to form healthy habits in your 20s - The New York Times,this article is part of a series aimed at helping you navigate life’s opportunities and challenges. what else should we write about? contact us: smarterliving@nytimes. com. when you woke up this m...
17,17,9,23.43,"company, year, million, business, market, percent, according, money, price, number, billion, job, also, new, investor, report, cost, study, economy, since","Modi’s Cash Ban Brings Pain, but Corruption-Weary India Grits Its Teeth - The New York Times","mumbai, india — it was a bold and risky gamble by prime minister narendra modi of india that quickly seemed to backfire. the announcement of a ban on the largest currency bills circulating in i..."
392,392,1,38.94,"people, like, get, say, would, think, thing, time, know, going, way, want, go, could, make, even, really, see, back, much","By Announcing New Jobs, Corporations Help Themselves Too - The New York Times","donald j. trump won’t be sworn in until friday but big business has already thrown him a veritable inaugural ball. a series of companies, among them ford, lockheed martin, amazon and sprint, hav..."
503,503,4,37.39,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",G.O.P. Senators Hope to Speed Trump Cabinet Confirmations - The New York Times,"washington — senate republicans will move this week to speed the confirmation of president trump’s cabinet, an effort that has been stymied by the combination of lax preparation by mr. trump’s ..."
681,681,4,33.36,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",Duterte’s Free Birth-Control Order Is Latest Skirmish With Catholic Church - The New York Times,"manila — when lizel torreras, 35, became pregnant with her third child, she mixed a tincture of bitter herbs and mahogany bark, a home remedy said to induce abortion. her husband, who worked as..."
733,733,3,23.21,"trump, president, obama, would, white_house, russia, country, administration, american, mr, russian, leader, also, united_state, donald_trump, official, former, meeting, government, deal",Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times,"paris — france chose an idealistic, traditional candidate in sunday’s primary to represent the socialist and parties in the presidential election this spring. the candidate, benoît hamon, 4..."
906,906,1,59.28,"people, like, get, say, would, think, thing, time, know, going, way, want, go, could, make, even, really, see, back, much",Cyprus Fears Russian Meddling in Its Settlement Talks - The New York Times,"nicosia, cyprus — as the united nations geared up for negotiations that it declared the “best and last chance” to unite cyprus after more than four decades of acrimonious division, russia’s amb..."
996,996,7,79.42,"investigation, report, official, case, information, time, email, according, fbi, also, statement, told, former, comey, evidence, reported, cnn, intelligence, would, charge",Mother Jones Is Named Magazine of the Year - The New York Times,"mother jones was named magazine of the year on tuesday for finding “new ways to engage audiences and contin

In [36]:
corpus_topic_df.groupby('Dominant Topic').apply(lambda topic_set:
                                            (topic_set.sort_values(by=['Contribution %'],
                                                   ascending=False).iloc[0]))

,Document,Dominant Topic,Contribution %,Topic Desc,Title,Paper
Dominant Topic,,,,,,
1,9997,1,96.16,"people, like, get, say, would, think, thing, time, know, going, way, want, go, could, make, even, really, see, back, much",Netanyahu: U.S.-Israel Alliance is ’About to Get Stronger’,ynetnews reports: prime minister benjamin netanyahu promised to strengthen the alliance between the us and israel on monday afternoon before taking off to washington for his first meeting with pre...
2,35668,2,94.07,"trump, clinton, campaign, republican, donald_trump, say, election, candidate, hillary_clinton, voter, party, people, vote, state, president, cruz, democrat, would, poll, think","Six dead, 10 injured after van crash in Virginia","(cnn) six people were killed in a van accident early saturday on a highway in virginia, state police said. the van ran off and came back onto the road. it hit a car that spun out of control, poli..."
3,45729,3,96.55,"trump, president, obama, would, white_house, russia, country, administration, american, mr, russian, leader, also, united_state, donald_trump, official, former, meeting, government, deal",Here’s the Hillary Clinton campaign’s memo spinning her New Hampshire loss,’the campaign of democratic presidential candidate hillary clinton released a memo downplaying the importance of the four early voting states after she to us sen. bernie sanders ( ).’ ’sanders wa...
4,44533,4,98.42,"attack, people, isi, country, government, group, muslim, syria, also, force, killed, military, city, war, many, official, year, security, turkey, two",Trump just scored a big win with the conservatives who killed his healthcare bill,’ ’ ’ the conservative house freedom caucus on wednesday reversed course and said it was on board with the latest iteration of a republican plan to repeal and replace obamacare. ’ ” to the ...
5,46978,5,95.19,"police, city, family, two, officer, people, home, day, man, child, according, told, three, shooting, cnn, time, car, state, student, death",5 good reasons to start a business before 40,’almost all of us think about starting a business at some point. the thought enters our minds when we come up with an ingenious way to stop an ice cream cone from dripping.’ ’or we get a job and r...
6,42361,6,91.92,"would, state, law, republican, american, bill, people, policy, president, house, right, issue, also, plan, could, senate, government, federal, gop, court",China: South China Sea island building ’almost complete’,"hong kong (cnn) china’s foreign ministry says it will soon complete controversial land reclamation on some islands and reefs in disputed waters in the south china sea. however, the ministry also ..."
7,167,7,98.46,"investigation, report, official, case, information, time, email, according, fbi, also, statement, told, former, comey, evidence, reported, cnn, intelligence, would, charge",Golden Globe Award Winners 2017: The Complete List - The New York Times,"here are the movies and tv shows that received the top awards. best picture, drama: “moonlight” best picture, comedy or musical: “la la land” actress, drama: isabelle huppert, “elle” actor, drama:..."
8,24339,8,95.43,"woman, show, year, also, time, black, life, new, first, like, story, white, film, wrote, star, book, people, movie, made, family",Complaints that Hillary Clinton Lied Under Oath Are ’Pending’ at the FBI - Breitbart,fbi director james comey confirmed wednesday that the fbi has received referrals to investigate hillary clinton for allegedly lying under oath to trey gowdy’s select committee on benghazi. [comey ...
9,26743,9,85.63,"company, year, million, business, market, percent, according, money, price, number, billion, job, also, new, investor, report, cost, study, economy, since","94,391,000 Americans Out of the Labor Force in August - Breitbart","the united states labor force participation rate rose by 58, 000 in august, according to statistics released by the labor department on friday. [that leaves a total of 94, 391, 000 a